<a href="https://colab.research.google.com/github/asgarali429/machine-learning-projects/blob/main/Search_Queries_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Queries.csv to Queries.csv


In [10]:
import pandas  as pd
from collections import Counter
import re
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

queries_df = pd.read_csv("Queries.csv")
queries_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


In [11]:
queries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [12]:
# queries_df['CTR']=queries_df['CTR'].str.replace('%','').astype('float')/100
queries_df['CTR'] = queries_df['CTR'].str.rstrip('%').astype('float') / 100

In [13]:
queries_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,0.3583,1.61
1,thecleverprogrammer,2809,3456,0.8128,1.02
2,python projects with source code,2077,73380,0.0283,5.94
3,classification report in machine learning,2012,4959,0.4057,1.28
4,the clever programmer,1931,2528,0.7638,1.09


In [15]:
# Function to clean and split the queries into words
def clean_and_split(queries):
  words = re.findall(r'\b[a-zA-z]+\b',queries.lower())
  return words

In [22]:
# Split each query into words and count the frequency of each word
word_counts = Counter()
for querie in queries_df['Top queries']:
  word_counts.update(clean_and_split(querie))


In [23]:
word_freq_df = pd.DataFrame(word_counts.most_common(20),columns=['word','frequency'])

In [24]:
fig = px.bar(word_freq_df,x='word',y='frequency',title = 'top 20  word in search squeries')
fig.show()

In [25]:
# Top queries by Clicks and Impressions
top_queries_clicks_vis = queries_df.nlargest(10, 'Clicks')[['Top queries', 'Clicks']]
top_queries_impressions_vis = queries_df.nlargest(10, 'Impressions')[['Top queries', 'Impressions']]

# Plotting
fig_clicks = px.bar(top_queries_clicks_vis, x='Top queries', y='Clicks', title='Top Queries by Clicks')
fig_impressions = px.bar(top_queries_impressions_vis, x='Top queries', y='Impressions', title='Top Queries by Impressions')
fig_clicks.show()
fig_impressions.show()

In [26]:
# Queries with highest and lowest CTR
top_ctr_vis = queries_df.nlargest(10, 'CTR')[['Top queries', 'CTR']]
bottom_ctr_vis = queries_df.nsmallest(10, 'CTR')[['Top queries', 'CTR']]

# Plotting
fig_top_ctr = px.bar(top_ctr_vis, x='Top queries', y='CTR', title='Top Queries by CTR')
fig_bottom_ctr = px.bar(bottom_ctr_vis, x='Top queries', y='CTR', title='Bottom Queries by CTR')
fig_top_ctr.show()
fig_bottom_ctr.show()

In [28]:
correlation_matrix = queries_df[['CTR','Clicks','Impressions','Position']].corr()
fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix')
fig_corr.show()

In [31]:
from sklearn.ensemble import IsolationForest

feature = queries_df[['CTR','Clicks','Impressions','Position']]

iso_forest = IsolationForest(n_estimators=100,contamination = 0.01)

iso_forest.fit(feature)

queries_df['Anomaly'] = iso_forest.predict(feature)

# Filtering out the anomalies
anomalies = queries_df[queries_df['Anomaly'] == -1]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



In [32]:
print(anomalies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']])

                                  Top queries  Clicks  Impressions     CTR  \
0                 number guessing game python    5223        14578  0.3583   
1                         thecleverprogrammer    2809         3456  0.8128   
2            python projects with source code    2077        73380  0.0283   
3   classification report in machine learning    2012         4959  0.4057   
4                       the clever programmer    1931         2528  0.7638   
7                 python turtle graphics code    1455        13585  0.1071   
15                 rock paper scissors python    1111        35824  0.0310   
21                      classification report     933        39896  0.0234   
34                   machine learning roadmap     708        42715  0.0166   
82                                   r2 score     367        56322  0.0065   

    Position  
0       1.61  
1       1.02  
2       5.94  
3       1.28  
4       1.09  
7       4.60  
15      7.19  
21      7.53  
34    